In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
 
import re
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/ML Projects/Google AI4Code/modules')
import pipeline, data_cleaner

Mounted at /content/drive


In [ ]:
data_path = '/content/drive/MyDrive/Colab Notebooks/ML Projects/Google AI4Code/data/unprocessed_kaggle_extra.zip'
check_path = '/content/unprocessed_kaggle_extra.zip'

pipeline.unzip_files(data_path, '/content', check_path)
kaggle_extra_data = pd.read_csv('/content/data.csv')

Unzipping files:   0%|          | 0/1 [00:00<?, ?it/s]


 Done unzipping data to disk path.


In [ ]:
def create_ids(length, count, prefix):
  return [prefix + str(i) for i in range(count, count+length)]

save_path = '/content/Kaggle Extra'
os.makedirs(save_path)

empty_text_pattern = re.compile('^[ \t\n]*$')

count = 0
doc_num = 0

for id in tqdm(kaggle_extra_data['notebook_id'].unique(), desc='Saving Kaggle Notebooks to CSV'):
  current_doc = kaggle_extra_data[kaggle_extra_data['notebook_id'] == id]

  current_doc = current_doc.drop(columns=['pct_rank', 'notebook_id', 'rank'])
  current_doc['source'] = current_doc['source'].replace(empty_text_pattern, np.nan)
  current_doc = current_doc.dropna()
  current_doc = current_doc.reset_index(drop=True)
  current_doc['rank'] = current_doc.index
  current_doc = current_doc.sort_values(by=['cell_type', 'rank'])
  
  value_counts = current_doc['cell_type'].value_counts()
  try:
    code_count = value_counts['code']
    md_count = value_counts['markdown']
  except:
    continue

  # Shuffle markdowns
  md_rows = current_doc[code_count:]
  md_rows = md_rows.sample(frac=1)
  current_doc[code_count:] = md_rows

  # Create cell_ids
  code_ids = create_ids(code_count, count, prefix='code_id_')
  md_ids = create_ids(md_count, count, prefix='md_id_')
  current_doc['cell_id'] = code_ids + md_ids

  if md_count >= 1 and code_count >= 1:
    current_doc.to_csv(save_path + f'/kag_data_{doc_num}.csv', index=False)
    count += len(current_doc)
    doc_num += 1

Saving Kaggle Notebooks to CSV:   0%|          | 0/168787 [00:00<?, ?it/s]

In [ ]:
# Check formatting
current_doc

,source,cell_type,rank,cell_id
0,Importing required packages.\nimport os\n impo...,code,1,code_id_1726533
1,Set up code checking\nfrom learntools.core imp...,code,2,code_id_1726534
2,"print the folder name, so it can be added to t...",code,3,code_id_1726535
3,Loading dataset\nwine = pd.read_csv('../input/...,code,4,code_id_1726536
4,checking a first few lines of the dataset\nwin...,code,5,code_id_1726537
...,...,...,...,...
72,We are going to look at few models and compare...,markdown,23,md_id_1726546
73,"In this notebook, we will first perform some p...",markdown,0,md_id_1726547
74,We will now see how to choose the best value u...,markdown,38,md_id_1726548
75,4. Stochastic Gradient Descent Classifier\n###...,markdown,52,md_id_1726549


In [ ]:
# kaggle_orders = {}

# for data_path in tqdm(kaggle_paths, desc='Loading Kaggle CSV Files'):
#   data = pd.read_csv(data_path, index_col='cell_id') 
#   id = data_path.stem
#   orders = list(data.sort_values(['rank']).index)
#   kaggle_orders[id] = orders

# kaggle_orders = pd.Series(kaggle_orders)
# kaggle_orders.to_json('/content/drive/MyDrive/Colab Notebooks/ML Projects/Google AI4Code/data/kag_orders.json')